Sam Vuong week 6 homework

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import csv


#importing weather api key
from config import weather_api_key
#import city dataset
from citipy import citipy


### Generate city list

In [51]:
#Generating unique city list

# generating city list (random coordinates), code taken from citypi github https://github.com/wingchen/citipy
city = citipy.nearest_city(22.99, 120.21)
city.city_name 
city.country_code


city_list = []
long_temp = 0
lat_temp = 0

while len(city_list) < 500:

    #creating random numbers based on min and max lat and long, in 2 decimal places as per the sample documentation 
    long_temp = round(np.random.uniform(-180, 180), 2)
    lat_temp = round(np.random.uniform(-90, 90), 2)

    #using nearest city method to generate a random city, then make sure it's not already on the list, then appending it to the city list
    city_temp = citipy.nearest_city(lat_temp, long_temp).city_name
    country_temp = citipy.nearest_city(lat_temp, long_temp).country_code

    if city_temp not in city_list:
        city_list.append([city_temp,country_temp])


city_list[1][0]

'jamestown'

In [23]:
city_weather_df

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': 18.296, 'lat': 57.6409}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 294.26, 'feels_like': 293.99, 'temp_m...",10000.0,"{'speed': 8.75, 'deg': 200}",{'all': 0},1.654954e+09,"{'type': 1, 'id': 1796, 'country': 'SE', 'sunr...",7200.0,2662689.0,Visby,200.0


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [65]:
#Weather query builder
# pandas can read json and make it into a dataframe, so i am going to create a dataframe of all 500 cities info then converting it to a csv

url = "https://api.openweathermap.org/data/2.5/weather?"


#varaible create, making a dictionary on these columns:  City	Lat	Lng	Max Temp	Humidity	Cloudiness	Wind Speed	Country	Date
city_count = 0


d_city = []
d_lat = []
d_lng = []
d_max_temp = []
d_humidity = []
d_cloudiness = []
d_wind_speed = []
d_country = []
d_date = []


#writing to txt 
f = open("report.txt", "w")

# for every city in the list, query the api, and then add the resulting json values to the appropriate list
for x in city_list:
    city_count += 1

    f.write(f"Processing record {city_count} | {x[0]} \n")
    query_url = url  + "q=" + x[0] + "&appid=" + weather_api_key + "&units=metric"
    weather_response = requests.get(query_url)
    print(weather_response)

    try:
       
        weather_json = weather_response.json()
        
        #appending results to list , putting the json variables first for error checking
        
        d_lat.append(weather_json["coord"]["lat"])
        d_lng.append(weather_json["coord"]["lon"])
        d_max_temp.append(weather_json["main"]["temp_max"])
        d_humidity.append(weather_json["main"]["humidity"])
        d_cloudiness.append(weather_json["clouds"]["all"])
        d_wind_speed.append(weather_json["wind"]["speed"])
        d_date.append(weather_json["dt"])
        d_city.append(x[0])
        d_country.append(x[1])
        


    except KeyError:   # this will catch cities that are not found because when we try to add the resulting json (non-existant) to the list we will get a keyerror
        f.write(f" Not found: {x[0]} Skipping \n")

        
    
f = open("report.txt", "r")
print(f.read())


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

### creating csv and dataframe of the results

In [69]:
city_weather_dict = {

    'City' : d_city,
    'Lat' : d_lat,
    'Lng' : d_lng,
    'Max Temp' : d_max_temp,
    'Humidity' : d_humidity,
    'Cloudiness' : d_cloudiness,
    'Wind Speed' : d_wind_speed,
    'Country' : d_country,
    'Date' : d_date

}

city_weather_df = pd.DataFrame(city_weather_dict)
city_weather_df.to_csv('city_weather.csv') 
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.1500,-70.9167,7.06,93,100,2.57,cl,1654957287
1,jamestown,42.0970,-79.2353,20.40,63,100,0.89,sh,1654957472
2,bluff,-46.6000,168.3333,1.00,82,87,5.19,nz,1654957474
3,shenavan,40.0549,43.9288,28.32,34,75,7.20,am,1654957475
4,medvedovskaya,45.4531,39.0258,31.12,34,58,4.50,ru,1654957476
...,...,...,...,...,...,...,...,...,...
458,kyra,49.5776,111.9753,12.08,93,100,7.13,ru,1654958032
459,pisco,-13.7000,-76.2167,15.57,100,100,1.03,pe,1654957570
460,chokurdakh,70.6333,147.9167,9.12,55,11,4.76,ru,1654958034
461,vestmannaeyjar,63.4427,-20.2734,11.55,82,40,5.14,is,1654958037


In [59]:
#Weather query builder Processing Record 1 of Set 1 | butaritari



weather_json["dt"]
d_city
weather_json

{'coord': {'lon': -81.2719, 'lat': -4.5772},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 19.42,
  'feels_like': 19.56,
  'temp_min': 19.42,
  'temp_max': 19.42,
  'pressure': 1014,
  'humidity': 82},
 'visibility': 10000,
 'wind': {'speed': 7.72, 'deg': 150},
 'clouds': {'all': 100},
 'dt': 1654956687,
 'sys': {'type': 1,
  'id': 8685,
  'country': 'PE',
  'sunrise': 1654946943,
  'sunset': 1654989639},
 'timezone': -18000,
 'id': 3691582,
 'name': 'Talara',
 'cod': 200}